*Read data form bronze layer*

In [0]:
%sql
SELECT * FROM bara_slaes_project.bronze.erp_cust_az12 LIMIT 5;

In [0]:
df=spark.table("bara_slaes_project.bronze.erp_cust_az12")

---
*Data Transformation*

In [0]:
# *Rename Cols*
rename_mapping = {
    'CID':'Customer_ID',
    'BDATE':'Date_Of_Birth',
    'GEN':'Gender',
}

df = df.withColumnsRenamed(rename_mapping)

In [0]:
## Trim
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import *
for field in df.schema.fields:
    if isinstance(field.dataType,StringType):
        df=df.withColumn(field.name,trim(col(field.name)))

In [0]:
#fill Null Values
df=df.fillna(value="Unknown",subset=["Gender"])
#Replace Values
Gender_replacements = {
    "F": "Female",
    "M": "Male",
    "": "Unknown"
}
df=df.replace(Gender_replacements,subset=["Gender"])

---
*Write into Silver layer*

In [0]:
df.write.mode("overwrite")\
    .format("Delta")\
        .saveAsTable("bara_slaes_project.silver.erp_customer_info")

In [0]:
%sql
SELECT * FROM bara_slaes_project.silver.erp_customer_info LIMIT 5;